<a href="https://colab.research.google.com/github/Mohammad-Rahmdel/Happy_Face_Recognition/blob/master/Happy_model_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1ZEO2AbXBSQvXAjrHvwogH-h7O9E95Xrx'})
download.GetContentFile('kt_utils.py')

!mkdir datasets
download = drive.CreateFile({'id': '1zsZa_qtvl_WeCJLGFOrPXqqya6kMz63L'})
download.GetContentFile('datasets/train_happy.h5')
download = drive.CreateFile({'id': '1ODfRLD-QlXtOT8w6tQZcLcmg-60AZMRM'})
download.GetContentFile('datasets/test_happy.h5')


In [19]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras import regularizers



X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

# print ("number of training examples = " + str(X_train.shape[0]))
# print ("number of test examples = " + str(X_test.shape[0]))
# print ("X_train shape: " + str(X_train.shape))
# print ("Y_train shape: " + str(Y_train.shape))
# print ("X_test shape: " + str(X_test.shape))
# print ("Y_test shape: " + str(Y_test.shape))


input_shape = X_train.shape[1]
X_input = Input(shape= (input_shape,input_shape,3))
X = ZeroPadding2D((3, 3))(X_input)


# CONV -> Batch Normalization -> RELU Block applied to X
# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
lambd = 3e-6
X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', kernel_regularizer=regularizers.l2(lambd))(X)  # n_H = n_W = 7  n_C = 32
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool')(X)

# FLATTEN X (means convert it to a vector) + FULLYCONNECTED
# X = Dropout(0.9)(X)
X = Flatten()(X)
X = Dense(1, activation='sigmoid', name='fc')(X)
# X = Dropout(1)(X)

# Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
model = Model(inputs = X_input, outputs = X, name='HappyModel')


"""
You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
    1. Create the model by calling the function above
    2. Compile the model by calling model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])
    3. Train the model on train data by calling model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)
    4. Test the model on test data by calling model.evaluate(x = ..., y = ...)
"""

model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(x = X_train, y = Y_train, epochs = 30, batch_size = 32, shuffle=True)
# model.summary()
# # plot_model(model)

preds = model.evaluate(X_train, Y_train)
print ("Test Accuracy = " + str(preds[1]))
preds = model.evaluate(X_test,Y_test)
print ("Test Accuracy = " + str(preds[1]))


Epoch 1/30
600/600 [==============================] - 1s 2ms/step - loss: 1.6660 - acc: 0.6583
Epoch 2/30
600/600 [==============================] - 0s 291us/step - loss: 0.4181 - acc: 0.8383
Epoch 3/30
600/600 [==============================] - 0s 270us/step - loss: 0.3385 - acc: 0.8617
Epoch 4/30
600/600 [==============================] - 0s 256us/step - loss: 0.1570 - acc: 0.9467
Epoch 5/30
600/600 [==============================] - 0s 266us/step - loss: 0.1159 - acc: 0.9583
Epoch 6/30
600/600 [==============================] - 0s 261us/step - loss: 0.0818 - acc: 0.9750
Epoch 7/30
600/600 [==============================] - 0s 251us/step - loss: 0.0703 - acc: 0.9767
Epoch 8/30
600/600 [==============================] - 0s 260us/step - loss: 0.0814 - acc: 0.9783
Epoch 9/30
600/600 [==============================] - 0s 268us/step - loss: 0.0640 - acc: 0.9867
Epoch 10/30
600/600 [==============================] - 0s 255us/step - loss: 0.0470 - acc: 0.9867
Epoch 11/30
600/600 [==========